<a href="https://colab.research.google.com/github/teteumac/Acoplamento-Quartico-Anomalo/blob/main/BDT_Output_Multiclass_DataDriven.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!python3 -m pip install coffea mplhep
!pip install matplotlib==3.1.3

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import mplhep as hep
from pandas import set_option
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import fbeta_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
import h5py
from sklearn.metrics import auc,make_scorer,fbeta_score,precision_score,recall_score,accuracy_score,log_loss,roc_auc_score,classification_report,f1_score,confusion_matrix,roc_curve,precision_recall_curve,average_precision_score
import math
from joblib import dump,load


In [ ]:
data_set_back_multirp = open_file_DD( PATH + 'DataDriven_Background_multiRP.h5' )


In [ ]:
PATH = '/content/gdrive/My Drive/Colab Notebooks/output_for_LGBM/'
plt.style.use(hep.style.ROOT)

def open_file_MC( file ):
    df = None
    with h5py.File( file , 'r' ) as f:
        dset = f[ 'dados' ]
        array = np.array( dset ) 
        array_cut = (array[:,0] > 600) #& (array[:,1] >= 200) & (array[:,2] >= 2) & (array[:,3] >= 2) & (array[:,4] >= 200) & (array[:,5] <= 2.4) & (array[:,7] <=0.6) & (array[:,8] >= 40) & (array[:,9] >= 53)  & (array[:,10] <= 2.4)
        DataSet_ = array[array_cut]
        dataframe = pd.DataFrame( DataSet_ , columns = ['Mww', 'Pt_W_lep', 'dPhi_Whad_Wlep', 'dPhi_jatos_MET', 'jetAK8_pt','jetAK8_eta', 'jetAK8_prunedMass',
'jetAK8_tau21', 'METPt', 'muon_pt', 'muon_eta', 'ExtraTracks', 'PUWeight', 'Yww', 'xi1', 'xi2', 'Mx', 'Yx','Mww/Mx', 'Yww_Yx', 'weight'] )
        dataframe['Acoplanaridade_Whad_Wlep'] = 1 - dataframe['dPhi_Whad_Wlep'].abs()/math.pi
        dataframe['Acoplanaridade_jatos_MET'] = 1 - dataframe['dPhi_jatos_MET'].abs()/math.pi 
        return dataframe

def open_file_DD( file ):
    df = None
    with h5py.File( file , 'r' ) as f:
        dset = f[ 'dados' ]
        array = np.array( dset ) 
        array_cut = (array[:,0] > 600) #& (array[:,1] >= 200) & (array[:,2] >= 2) & (array[:,3] >= 2) & (array[:,4] >= 200) & (array[:,5] <= 2.4) & (array[:,7] <=0.6) & (array[:,8] >= 40) & (array[:,9] >= 53)  & (array[:,10] <= 2.4)
        DataSet_ = array[array_cut]
        dataframe = pd.DataFrame( DataSet_ , columns = ['Mww', 'Pt_W_lep', 'dPhi_Whad_Wlep', 'dPhi_jatos_MET', 'jetAK8_pt','jetAK8_eta', 'jetAK8_prunedMass',
'jetAK8_tau21', 'METPt', 'muon_pt', 'muon_eta', 'ExtraTracks', 'PUWeight', 'Yww', 'xi1', 'xi2', 'Mx', 'Yx','Mww/Mx', 'Yww_Yx', 'weight'] )
        dataframe['Acoplanaridade_Whad_Wlep'] = 1 - dataframe['dPhi_Whad_Wlep'].abs()/math.pi
        dataframe['Acoplanaridade_jatos_MET'] = 1 - dataframe['dPhi_jatos_MET'].abs()/math.pi 
        return dataframe         

def open_file_Data( file ):
    df = None
    with h5py.File( file , 'r' ) as f:
        dset = f[ 'dados' ]
        array = np.array( dset ) 
        array_cut = (array[:,0] > 600) #& (array[:,1] >= 200) & (array[:,2] >= 2) & (array[:,3] >= 2) & (array[:,4] >= 200) & (array[:,5] <= 2.4) & (array[:,7] <=0.6) & (array[:,8] >= 40) & (array[:,9] >= 53)  & (array[:,10] <= 2.4)
        DataSet_ = array[array_cut]
        dataframe = pd.DataFrame( DataSet_ , columns = ['Mww', 'Pt_W_lep', 'dPhi_Whad_Wlep', 'dPhi_jatos_MET', 'jetAK8_pt','jetAK8_eta', 'jetAK8_prunedMass',
'jetAK8_tau21', 'METPt', 'muon_pt', 'muon_eta', 'ExtraTracks', 'Yww', 'xi1', 'xi2', 'Mx', 'Yx','Mww/Mx', 'Yww_Yx', 'weight'] )
        dataframe['Acoplanaridade_Whad_Wlep'] = 1 - dataframe['dPhi_Whad_Wlep'].abs()/math.pi
        dataframe['Acoplanaridade_jatos_MET'] = 1 - dataframe['dPhi_jatos_MET'].abs()/math.pi 
        return dataframe       


select_columns = ['Mww', 'Pt_W_lep', 'Acoplanaridade_Whad_Wlep', 'Acoplanaridade_jatos_MET', 'jetAK8_pt','jetAK8_eta', 'jetAK8_prunedMass', 'jetAK8_tau21', 'METPt', 'muon_pt', 'muon_eta', 'ExtraTracks', 'Mx',  'Mww/Mx']

SM =       open_file_MC( PATH + 'DataSet_SM_multiRP.h5')
ANOMALO1 = open_file_MC( PATH + 'DataSet_ANOMALO1_multiRP.h5')
ANOMALO2 = open_file_MC( PATH + 'DataSet_ANOMALO2_multiRP.h5')
ANOMALO3 = open_file_MC( PATH + 'DataSet_ANOMALO3_multiRP.h5')
ANOMALO4 = open_file_MC( PATH + 'DataSet_ANOMALO4_multiRP.h5')
ANOMALO5 = open_file_MC( PATH + 'DataSet_ANOMALO5_multiRP.h5')
ANOMALO6 = open_file_MC( PATH + 'DataSet_ANOMALO6_multiRP.h5')
ANOMALO7 = open_file_MC( PATH + 'DataSet_ANOMALO7_multiRP.h5')
ANOMALO8 = open_file_MC( PATH + 'DataSet_ANOMALO8_multiRP.h5')

label_signal1  = pd.DataFrame( [1]*len( ANOMALO1 ) )
label_signal2  = pd.DataFrame( [1]*len( ANOMALO2 ) )
label_signal3  = pd.DataFrame( [1]*len( ANOMALO3 ) )
label_signal4  = pd.DataFrame( [1]*len( ANOMALO4 ) )
label_signal5  = pd.DataFrame( [1]*len( ANOMALO5 ) )
label_signal6  = pd.DataFrame( [1]*len( ANOMALO6 ) )
label_signal7  = pd.DataFrame( [1]*len( ANOMALO7 ) )
label_signal8  = pd.DataFrame( [1]*len( ANOMALO8 ) )
label_signalSM = pd.DataFrame( [2]*len( SM ) )

SM = pd.concat( [ SM, label_signalSM ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Standard Model --> ', SM.shape)

ANOMALO1 = pd.concat( [ ANOMALO1, label_signal1 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 1 --> ', ANOMALO1.shape)

ANOMALO2 = pd.concat( [ ANOMALO2, label_signal2 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 2 --> ', ANOMALO2.shape)

ANOMALO3 = pd.concat( [ ANOMALO3, label_signal3 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 3 --> ', ANOMALO3.shape)

ANOMALO4 = pd.concat( [ ANOMALO4, label_signal4 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 4 --> ', ANOMALO4.shape)

ANOMALO5 = pd.concat( [ ANOMALO5, label_signal5 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 5 --> ', ANOMALO5.shape)

ANOMALO6 = pd.concat( [ ANOMALO6, label_signal6 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 6 --> ', ANOMALO6.shape)

ANOMALO7 = pd.concat( [ ANOMALO7, label_signal7 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 7 --> ', ANOMALO7.shape)

ANOMALO8 = pd.concat( [ ANOMALO8, label_signal8 ], axis = 1 ).rename(columns={0: 'label'})
print('Shape for Anomalo 8 --> ', ANOMALO8.shape)

data_set_back_multirp = open_file_DD( PATH + 'DataDriven_Background_multiRP.h5' )

label_back = pd.DataFrame( [0]*len( data_set_back_multirp ) )
data_set_back_multirp = pd.concat( [ data_set_back_multirp, label_back ], axis = 1 ).rename(columns={0: 'label'})
data_set_back_multirp

Dataset_Signal_Back1  = pd.concat( [ ANOMALO1 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back2  = pd.concat( [ ANOMALO2 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back3  = pd.concat( [ ANOMALO3 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back4  = pd.concat( [ ANOMALO4 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back5  = pd.concat( [ ANOMALO5 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back6  = pd.concat( [ ANOMALO6 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back7  = pd.concat( [ ANOMALO7 , data_set_back_multirp, SM  ], axis = 0, sort = False )
Dataset_Signal_Back8  = pd.concat( [ ANOMALO8 , data_set_back_multirp, SM  ], axis = 0, sort = False )

data_set_dados_multirp = open_file_Data( PATH + 'DataSet_dados_multiRP.h5' )
data_set_dados_multirp = data_set_dados_multirp[select_columns]

from sklearn.model_selection import train_test_split

test_size = 0.35
DataSet_Train1_, DataSet_Test1_ = train_test_split( Dataset_Signal_Back1, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back1.label )
DataSet_Train2_, DataSet_Test2_ = train_test_split( Dataset_Signal_Back2, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back2.label )
DataSet_Train3_, DataSet_Test3_ = train_test_split( Dataset_Signal_Back3, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back3.label )
DataSet_Train4_, DataSet_Test4_ = train_test_split( Dataset_Signal_Back4, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back4.label )
DataSet_Train5_, DataSet_Test5_ = train_test_split( Dataset_Signal_Back5, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back5.label )
DataSet_Train6_, DataSet_Test6_ = train_test_split( Dataset_Signal_Back6, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back6.label )
DataSet_Train7_, DataSet_Test7_ = train_test_split( Dataset_Signal_Back7, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back7.label )
DataSet_Train8_, DataSet_Test8_ = train_test_split( Dataset_Signal_Back8, test_size = test_size, random_state = 42, stratify = Dataset_Signal_Back8.label )


DataSet_Train8 = DataSet_Train8_[select_columns] 
DataSet_Test8 = DataSet_Test8_[select_columns] 

DataSet_Train7 = DataSet_Train7_[select_columns] 
DataSet_Test7 = DataSet_Test7_[select_columns] 

DataSet_Train6 = DataSet_Train6_[select_columns] 
DataSet_Test6 = DataSet_Test6_[select_columns] 

DataSet_Train5 = DataSet_Train5_[select_columns] 
DataSet_Test5 = DataSet_Test5_[select_columns] 

DataSet_Train1 = DataSet_Train1_[select_columns] 
DataSet_Test1 = DataSet_Test1_[select_columns] 

DataSet_Train2 = DataSet_Train2_[select_columns] 
DataSet_Test2 = DataSet_Test2_[select_columns] 

DataSet_Train3 = DataSet_Train3_[select_columns] 
DataSet_Test3 = DataSet_Test3_[select_columns] 

DataSet_Train4 = DataSet_Train4_[select_columns] 
DataSet_Test4 = DataSet_Test4_[select_columns] 

y_train1 = DataSet_Train1_['label']
y_test1  = DataSet_Test1_['label']

y_train2 = DataSet_Train2_['label']
y_test2  = DataSet_Test2_['label']

y_train3 = DataSet_Train3_['label']
y_test3  = DataSet_Test3_['label']

y_train4 = DataSet_Train4_['label']
y_test4  = DataSet_Test4_['label']

y_train5 = DataSet_Train5_['label']
y_test5  = DataSet_Test5_['label']

y_train6 = DataSet_Train6_['label']
y_test6  = DataSet_Test6_['label']

y_train7 = DataSet_Train7_['label']
y_test7  = DataSet_Test7_['label']

y_train8 = DataSet_Train8_['label']
y_test8  = DataSet_Test8_['label']

print('--- Weight Anomalo 8 --- \n')
DataSet_Test8_weight_signal =  DataSet_Test8_[DataSet_Test8_['label']==1]['weight'] 
DataSet_Test8_weight_backgr =DataSet_Test8_[DataSet_Test8_['label']==0]['weight'] 
DataSet_TestSM8_weight_signal =  DataSet_Test8_[DataSet_Test8_['label']==2]['weight'] 

DataSet_Train8_weight_signal = DataSet_Train8_[DataSet_Train8_['label']==1]['weight']
DataSet_Train8_weight_backgr = DataSet_Train8_[DataSet_Train8_['label']==0]['weight'] 

DataSet_Test8_weight =  DataSet_Test8_['weight']

print( 'Shape Test Signal', DataSet_Test8_weight_signal.shape)
print( 'Shape Test Background', DataSet_Test8_weight_backgr.shape, '\n')

print('--- Weight Anomalo 7 --- \n')

DataSet_Test7_weight_signal =  DataSet_Test7_[DataSet_Test7_['label']==1]['weight'] 
DataSet_Test7_weight_backgr =  DataSet_Test7_[DataSet_Test7_['label']==0]['weight'] 
DataSet_TestSM7_weight_signal =  DataSet_Test7_[DataSet_Test7_['label']==2]['weight'] 

DataSet_Train7_weight_signal =  DataSet_Train7_[DataSet_Train7_['label']==1]['weight']
DataSet_Train7_weight_backgr =  DataSet_Train7_[DataSet_Train7_['label']==0]['weight'] 

DataSet_Test7_weight =  DataSet_Test7_['weight']

print('--- Weight Anomalo 6 --- \n')

DataSet_Test6_weight_signal =  DataSet_Test6_[DataSet_Test6_['label']==1]['weight'] 
DataSet_Test6_weight_backgr =  DataSet_Test6_[DataSet_Test6_['label']==0]['weight'] 
DataSet_TestSM6_weight_signal =  DataSet_Test6_[DataSet_Test6_['label']==2]['weight'] 

DataSet_Train6_weight_signal =  DataSet_Train6_[DataSet_Train6_['label']==1]['weight']
DataSet_Train6_weight_backgr =  DataSet_Train6_[DataSet_Train6_['label']==0]['weight'] 

DataSet_Test6_weight =  DataSet_Test6_['weight']

print( 'Shape Test Signal',  DataSet_Test7_weight_signal.shape)
print( 'Shape Test Background' , DataSet_Test7_weight_backgr.shape, '\n')

print('--- Weight Anomalo 5 --- \n')

DataSet_Test5_weight_signal =  DataSet_Test5_[DataSet_Test5_['label']==1]['weight'] 
DataSet_Test5_weight_backgr =  DataSet_Test5_[DataSet_Test5_['label']==0]['weight'] 
DataSet_TestSM5_weight_signal =  DataSet_Test5_[DataSet_Test5_['label']==2]['weight'] 

DataSet_Train5_weight_signal =  DataSet_Train5_[DataSet_Train5_['label']==1]['weight']
DataSet_Train5_weight_backgr =  DataSet_Train5_[DataSet_Train5_['label']==0]['weight'] 

DataSet_Test5_weight =  DataSet_Test5_['weight']

print( 'Shape Test Signal',  DataSet_Test7_weight_signal.shape)
print( 'Shape Test Background' , DataSet_Test7_weight_backgr.shape, '\n')

print('--- Weight Anomalo 1 --- \n')

DataSet_Test1_weight_signal =  DataSet_Test1_[DataSet_Test1_['label']==1]['weight'] 
DataSet_Test1_weight_backgr =  DataSet_Test1_[DataSet_Test1_['label']==0]['weight'] 
DataSet_TestSM1_weight_signal =  DataSet_Test1_[DataSet_Test1_['label']==2]['weight'] 

DataSet_Train1_weight_signal =  DataSet_Train1_[DataSet_Train1_['label']==1]['weight']
DataSet_Train1_weight_backgr = DataSet_Train1_[DataSet_Train1_['label']==0]['weight'] 

DataSet_Test1_weight =  DataSet_Test1_['weight']

print( 'Shape Test Signal', DataSet_Test1_weight_signal.shape)
print( 'Shape Test Background' , DataSet_Test1_weight_backgr.shape, '\n')

print('--- Weight Anomalo 2 --- \n')

DataSet_Test2_weight_signal =  DataSet_Test2_[DataSet_Test2_['label']==1]['weight'] 
DataSet_Test2_weight_backgr =  DataSet_Test2_[DataSet_Test2_['label']==0]['weight'] 
DataSet_TestSM2_weight_signal =  DataSet_Test2_[DataSet_Test2_['label']==2]['weight'] 

DataSet_Train2_weight_signal =  DataSet_Train2_[DataSet_Train2_['label']==1]['weight']
DataSet_Train2_weight_backgr = DataSet_Train2_[DataSet_Train2_['label']==0]['weight'] 

DataSet_Test2_weight =  DataSet_Test2_['weight']

print( 'Shape Test Signal', DataSet_Test2_weight_signal.shape)
print( 'Shape Test Background' , DataSet_Test2_weight_backgr.shape, '\n')

print('--- Weight Anomalo 3 --- \n')

DataSet_Test3_weight_signal =  DataSet_Test3_[DataSet_Test3_['label']==1]['weight'] 
DataSet_Test3_weight_backgr =  DataSet_Test3_[DataSet_Test3_['label']==0]['weight'] 
DataSet_TestSM3_weight_signal =  DataSet_Test3_[DataSet_Test3_['label']==2]['weight'] 

DataSet_Train3_weight_signal =  DataSet_Train3_[DataSet_Train3_['label']==1]['weight']
DataSet_Train3_weight_backgr = DataSet_Train3_[DataSet_Train3_['label']==0]['weight'] 

DataSet_Test3_weight =  DataSet_Test3_['weight']

print( 'Shape Test Signal', DataSet_Test3_weight_signal.shape)
print( 'Shape Test Background' , DataSet_Test3_weight_backgr.shape, '\n')

print('--- Weight Anomalo 4 --- \n')

DataSet_Test4_weight_signal = DataSet_Test4_[DataSet_Test4_['label']==1]['weight'] 
DataSet_Test4_weight_backgr = DataSet_Test4_[DataSet_Test4_['label']==0]['weight'] 
DataSet_TestSM4_weight_signal =  DataSet_Test4_[DataSet_Test4_['label']==2]['weight'] 

DataSet_Train4_weight_signal = DataSet_Train4_[DataSet_Train4_['label']==1]['weight']
DataSet_Train4_weight_backgr = DataSet_Train4_[DataSet_Train4_['label']==0]['weight'] 

DataSet_Test4_weight =  DataSet_Test4_['weight']

print( 'Shape Test Signal' , DataSet_Test4_weight_signal.shape)
print( 'Shape Test Background' , DataSet_Test4_weight_backgr.shape)

In [ ]:
from sklearn.preprocessing import label_binarize

def plot_precision_recall_curve( label_test, predicit, title, nome ):
    label_test = label_binarize( label_test, classes = [*range(3) ] )
    precision = dict()
    recall = dict()
    threshs = dict()
    fontsize = 18

    fig, axes = plt.subplots( 1, 2, figsize=(19,10) )

    for i in range(3):
        precision[i], recall[i], threshs[i] = precision_recall_curve( label_test[:,i], predicit[:, i])
        axes[0].plot( recall[i], precision[i], lw = 2, label = ' class {}'.format(i) )
    axes[0].set_xlabel( "Recall", fontsize = fontsize )
    axes[0].set_ylabel( "Precision", fontsize = fontsize )
    axes[0].legend( loc = "best", fontsize = 14 )
    axes[0].set_title( title)

    thresholds = np.concatenate( [ threshs[1], [1] ], axis = 0 )
    bidx = np.argmax( precision[1] * recall[1] )
    best_cut = threshs[1][bidx]
    print( '\n',' Best Cut ', best_cut,'\n' )

    axes[1].plot( thresholds , precision[1] * recall[1],  color = 'blue' )
    axes[1].plot( [ best_cut , best_cut ] , [-0.1,(precision[1] * recall[1]).max()*2] ,"-.r",label='Best Cut : {:2.5f}'.format(best_cut) )
    axes[1].plot( [-0.1,1.1] , [ ( precision[1]*recall[1] ).max(),( precision[1]*recall[1] ).max() ], ":g",label=r'Precision $\times$ Recall : {:2.5f}'.format( (precision[1]*recall[1]).max() ) )
    axes[1].set_ylabel( r'Precision $\times$ Recall', fontsize = fontsize )
    axes[1].set_xlabel(    'Thresholds', fontsize = fontsize    )
    axes[1].set_ylim( -0.1 , 1 )
    axes[1].set_xlim( -0.01 , 1 )
    axes[1].set_title( title )
    axes[1].legend( loc = "best", fontsize = 14)

def best_cut( label_test, predicit ):
    label_test = label_binarize( label_test, classes = [ *range(3) ] )
    precision = dict()
    recall = dict()
    threshs = dict()

    for i in range(3):
        precision[i], recall[i], threshs[i] = precision_recall_curve( label_test[:,i], predicit[:, i] )

    thresholds = np.concatenate( [ threshs[1], [1] ], axis = 0 )
    bidx = np.argmax( precision[1] * recall[1] )
    return threshs[1][bidx]

In [ ]:
def result_model(joblib, DataSet_Test, DataSet_Train, y_test, y_train):   
    model_ANOMALO = load( PATH + joblib )
    predict_ANOMALO = model_ANOMALO.predict_proba(DataSet_Test)
    predict_ANOMALO_train = model_ANOMALO.predict_proba(DataSet_Train)
    best_cut_ANOMALO = best_cut(y_train,predict_ANOMALO_train)
    best_cut_ANOMALO_train = best_cut(y_test,predict_ANOMALO)

    print( 'best cut', best_cut_ANOMALO )
    label_pred_ANOMALO = [np.argmax(line) for line in predict_ANOMALO > best_cut_ANOMALO]

    print('best_cut_test-->', best_cut_ANOMALO)
    print("Purity in test sample     : {:2.2f}%".format(100 * precision_score(y_test, predict_ANOMALO, average='weighted')))
    print("Efficiency in test sample : {:2.2f}%".format(100 * recall_score(y_test, predict_ANOMALO, average='weighted')))
    print("Accuracy in test sample   : {:2.2f}%".format(100 * accuracy_score(y_test, predict_ANOMALO, average='weighted')))
    print( "F1_score in test sample  : {:2.2f}%".format(100 * f1_score(y_test, predict_ANOMALO, average='weighted')))
    print("\n")
    print('best_cut_train-->', best_cut_ANOMALO_train)
    print("Purity in train sample     : {:2.2f}%".format(100 * precision_score(y_train, predict_ANOMALO_train, average='weighted')))
    print("Efficiency in train sample : {:2.2f}%".format(100 * recall_score(y_train, predict_ANOMALO_train, average='weighted')))
    print("Accuracy in train sample   : {:2.2f}%".format(100 * accuracy_score(y_train, predict_ANOMALO_train, average='weighted')))
    print( "F1_score in train sample  : {:2.2f}%".format(100 * f1_score(y_train, predict_ANOMALO_train, average='weighted')))
    return predict_ANOMALO, best_cut_ANOMALO

In [ ]:
from sklearn.preprocessing import label_binarize
def result( X_test, Y_test, X_train, Y_train, joblib, label, verbose = 1 ):
    print(' ---------------- ' + label + '----------------' )
    scaler = StandardScaler()
    X_scaled_test = scaler.fit_transform(X_test)
    X_scaled_train = scaler.fit_transform(X_train)

    model = load( PATH + joblib )
    
    predict_proba_test = model.predict_proba(X_scaled_test)  # calculate the probability
    predict_proba_train = model.predict_proba(X_scaled_train)  # calculate the probability

    label_test = label_binarize( Y_test, classes = [ *range(3) ] )
    precision_test = dict()
    recall_test = dict()
    threshs_test = dict()

    for i in range(3):
        precision_test[i], recall_test[i], threshs_test[i] = precision_recall_curve( label_test[:,i], predict_proba_test[:,i] )

    thresholds_test = np.concatenate( [ threshs_test[1], [1] ], axis = 0 )
    bidx_test = np.argmax( precision_test[1] * recall_test[1] )
    best_cut_test = threshs_test[1][bidx_test]
    print('best cut test--> ', best_cut_test)
    #y_preds_test = [ np.argmax(line) for line in predict_proba_test[:,1] > best_cut_test ]

    label_predict_test = model.predict(X_scaled_test)
    label_predict_train = model.predict(X_scaled_train)
    print('label_predict_test', pd.DataFrame(label_predict_test).value_counts())
    print('label_predict_train', pd.DataFrame(label_predict_train).value_counts())

    label_train = label_binarize( Y_train, classes = [ *range(3) ] )
    precision_train = dict()
    recall_train = dict()
    threshs_train = dict()

    for i in range(3):
        precision_train[i], recall_train[i], threshs_train[i] = precision_recall_curve( label_train[:,i], predict_proba_train[:,i] )

    thresholds_train = np.concatenate( [ threshs_train[1], [1] ], axis = 0 )
    bidx_train = np.argmax( precision_train[1] * recall_train[1] )
    best_cut_train = threshs_train[1][bidx_train]
    print('best cut train -->', best_cut_train)
    y_preds_train = [ np.argmax(line) for line in predict_proba_train > best_cut_train ]    

    if verbose == 1:
        print("Purity in test sample     : {:2.2f}%".format(100 * precision_score(Y_test, label_predict_test, average = 'weighted')))
        print("Efficiency in test sample : {:2.2f}%".format(100 * recall_score(Y_test, label_predict_test, average = 'weighted')))
        print("F1_score in test sample   : {:2.2f}%".format(100 * f1_score(Y_test, label_predict_test, average = 'weighted')))
        print('\n')
        print("Purity in train sample     : {:2.2f}%".format(100 * precision_score(Y_train, label_predict_train, average = 'weighted')))
        print("Efficiency in train sample : {:2.2f}%".format(100 * recall_score(Y_train, label_predict_train, average = 'weighted')))
        print("F1_score in train sample   : {:2.2f}%".format(100 * f1_score(Y_train, label_predict_train, average = 'weighted')))    

    return predict_proba_test[:,1], best_cut_test, label_predict_test   

In [ ]:
scaler = StandardScaler()
X_scaled_dados = scaler.fit_transform(data_set_dados_multirp)

In [ ]:
model_ANOMALO5 = load( PATH + 'LGBM_Multiclass_ANOMALO5_DataDriven.joblib' )
model_ANOMALO6 = load( PATH + 'LGBM_Multiclass_ANOMALO6_DataDriven.joblib' )
model_ANOMALO7 = load( PATH + 'LGBM_Multiclass_ANOMALO7_DataDriven.joblib' )
model_ANOMALO8 = load( PATH + 'LGBM_Multiclass_ANOMALO8_DataDriven.joblib' )

model_ANOMALO1 = load( PATH + 'LGBM_Multiclass_ANOMALO1_DataDriven.joblib' )
model_ANOMALO2 = load( PATH + 'LGBM_Multiclass_ANOMALO2_DataDriven.joblib' )
model_ANOMALO3 = load( PATH + 'LGBM_Multiclass_ANOMALO3_DataDriven.joblib' )
model_ANOMALO4 = load( PATH + 'LGBM_Multiclass_ANOMALO4_DataDriven.joblib' )

In [ ]:
DataSet_Test8.shape

In [ ]:
predict_ANOMALO8, best_cut_ANOMALO8, label_predict_test8 = result(DataSet_Test8, y_test8, DataSet_Train8, y_train8, 'LGBM_Multiclass_ANOMALO8_DataDriven.joblib', 'ANOMALO8' )

In [ ]:
predict_ANOMALO7, best_cut_ANOMALO7, label_predict_test7 = result(DataSet_Test7, y_test7, DataSet_Train7, y_train7, 'LGBM_Multiclass_ANOMALO7_DataDriven.joblib', 'ANOMALO7' )

In [ ]:
predict_ANOMALO6, best_cut_ANOMALO6, label_predict_test6 = result(DataSet_Test6, y_test6, DataSet_Train6, y_train6, 'LGBM_Multiclass_ANOMALO6_DataDriven.joblib', 'ANOMALO6' )

In [ ]:
predict_ANOMALO5, best_cut_ANOMALO5 = result(DataSet_Test5, y_test5, DataSet_Train5, y_train5, 'LGBM_Multiclass_ANOMALO5_DataDriven.joblib', 'ANOMALO5' )

In [ ]:
predict_ANOMALO4, best_cut_ANOMALO4 = result(DataSet_Test4, y_test4, DataSet_Train4, y_train4, 'LGBM_Multiclass_ANOMALO4_DataDriven.joblib', 'ANOMALO4' )

In [ ]:
predict_ANOMALO3, best_cut_ANOMALO3 = result(DataSet_Test3, y_test3, DataSet_Train3, y_train3, 'LGBM_Multiclass_ANOMALO3_DataDriven.joblib', 'ANOMALO3' )

In [ ]:
predict_ANOMALO2, best_cut_ANOMALO2 = result(DataSet_Test2, y_test2, DataSet_Train2, y_train2, 'LGBM_Multiclass_ANOMALO2_DataDriven.joblib', 'ANOMALO2' )

In [ ]:
predict_ANOMALO1, best_cut_ANOMALO1 = result(DataSet_Test1, y_test1, DataSet_Train1, y_train1, 'LGBM_Multiclass_ANOMALO1_DataDriven.joblib', 'ANOMALO1' )

In [ ]:
fig, ax = plt.subplots( 2, 2, figsize=(24,15) )
predict_dados = model_ANOMALO4.predict_proba(X_scaled_dados)[:,1]
bins = 25

hbgt_lgb = ax[0,0].hist( predict_ANOMALO4[ y_test4 == 0 ], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Data-Driven Background', weights = DataSet_Test4_weight_backgr/test_size )
ax[0,0].hist( predict_ANOMALO4[ y_test4 == 1 ], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{C}^{W}/\Lambda^{2}=8.0 \times 10^{-6}$', weights = DataSet_Test4_weight_signal/test_size, color = 'green' )
ax[0,0].hist( predict_ANOMALO4[ y_test4 == 2 ], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Modelo Padrão', weights = DataSet_TestSM4_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[0,0].plot( [ best_cut_ANOMALO4,best_cut_ANOMALO4],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut_ANOMALO4 ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut_ANOMALO4 ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[0,0].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut4 = predict_ANOMALO4[ y_test4 == 0 ] > best_cut_ANOMALO4 
n_events_signal_after_cut4 = predict_ANOMALO4[ y_test4 == 1 ] > best_cut_ANOMALO4
n_events_SM_after_cut4 = predict_ANOMALO4[ y_test4 == 2 ] > best_cut_ANOMALO4
n_eventos_Data_after_cut4 = predict_dados[ predict_dados >= best_cut_ANOMALO4]

print(" --------------- Anomalo 4 --------------- ")
print('Numero de eventos de background depois do corte -->', DataSet_Test4_weight_backgr[ n_events_back_after_cut4 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test4_weight_signal[ n_events_signal_after_cut4].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM4_weight_signal[ n_events_SM_after_cut4 ].sum() / test_size )


ax[0,0].text(best_cut_ANOMALO4+0.05 , 100, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[0,0].set_ylabel( "Events (Log Scale)" )
ax[0,0].set_title( "Mangueira Plot", fontsize = 16  )
ax[0,0].legend( loc = "upper center", fontsize = 16 )
ax[0,0].set_yscale( 'log' )
ax[0,0] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[0,0] )


predict_dados = model_ANOMALO1.predict_proba(X_scaled_dados)[:,1]
bins = 25

hbgt_lgb = ax[0,1].hist( predict_ANOMALO1[ y_test1 == 0 ], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Data-Drive Background', weights = DataSet_Test1_weight_backgr/test_size )
ax[0,1].hist( predict_ANOMALO1[ y_test1 == 1 ], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{C}^{W}/\Lambda^{2}=20 \times 10^{-6}$', weights = DataSet_Test1_weight_signal/test_size, color = 'green' )
ax[0,1].hist( predict_ANOMALO1[ y_test1 == 2 ], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Modelo Padrão', weights = DataSet_TestSM1_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[0,1].plot( [ best_cut_ANOMALO1,best_cut_ANOMALO1],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut_ANOMALO1 ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut_ANOMALO1] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[0,1].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut1 = predict_ANOMALO1[ y_test1 == 0 ] > best_cut_ANOMALO1
n_events_signal_after_cut1 = predict_ANOMALO1[ y_test1 == 1 ] > best_cut_ANOMALO1
n_events_SM_after_cut1 = predict_ANOMALO1[ y_test1 == 2 ] > best_cut_ANOMALO1
n_eventos_Data_after_cut1 = predict_dados[ predict_dados >= best_cut_ANOMALO1 ]

print(' --------------- Anomalo 1 --------------- ')
print('Numero de eventos de background depois do corte -->', DataSet_Test1_weight_backgr[ n_events_back_after_cut1 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test1_weight_signal[ n_events_signal_after_cut1].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM1_weight_signal[ n_events_SM_after_cut1 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut1 ) )


ax[0,1].text(best_cut_ANOMALO1 + 0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16, rotation = 90 )    
ax[0,1].set_title( "Mangueira Plot", fontsize = 16  )
ax[0,1].legend( loc = "upper center", fontsize = 16 )
ax[0,1].set_yscale( 'log' )
ax[0,1] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[0,1] )

predict_dados = model_ANOMALO3.predict_proba(X_scaled_dados)[:,1]
bins = 25

hbgt_lgb = ax[1,0].hist( predict_ANOMALO3[ y_test3 == 0 ], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Data-Driven Background', weights = DataSet_Test3_weight_backgr/test_size )
ax[1,0].hist( predict_ANOMALO3[ y_test3 == 1 ], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{C}^{W}/\Lambda^{2} = 5.0 \times 10^{-6} $', weights = DataSet_Test3_weight_signal/test_size, color = 'green' )
ax[1,0].hist( predict_ANOMALO3[ y_test3 == 2 ], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Modelo Padrão', weights = DataSet_TestSM3_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[1,0].plot( [ best_cut_ANOMALO3,best_cut_ANOMALO3],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut_ANOMALO3 ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut_ANOMALO3 ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[1,0].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Dados-2016', color = 'black' )

n_events_back_after_cut3 = predict_ANOMALO3[ y_test3 == 0 ] >= best_cut_ANOMALO3
n_events_signal_after_cut3 = predict_ANOMALO3[ y_test3 == 1 ] >= best_cut_ANOMALO3
n_events_SM_after_cut3 = predict_ANOMALO3[ y_test3 == 2 ] >= best_cut_ANOMALO3
n_eventos_Data_after_cut3 = predict_dados[ predict_dados >= best_cut_ANOMALO3 ]

print(' --------------- Anomalo 3 --------------- ')
print('Numero de eventos de background depois do corte -->', DataSet_Test3_weight_backgr[ n_events_back_after_cut3 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test3_weight_signal[ n_events_signal_after_cut3].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM3_weight_signal[ n_events_SM_after_cut3 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut3 ) )


ax[1,0].text(best_cut_ANOMALO3+0.05 , 2, 'Signal Region \n Blinded Data', color = 'red', fontsize = 16)    
ax[1,0].set_title( "Mangueira Plot", fontsize = 16  )
ax[1,0].legend( loc = "upper right", fontsize = 15  )
ax[1,0].set_yscale( 'log' )
ax[1,0] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[1,0] )

predict_dados = model_ANOMALO2.predict_proba(X_scaled_dados)[:,1]
bins = 25

hbgt_lgb = ax[1,1].hist( predict_ANOMALO2[ y_test2 == 0 ], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Data-Driven Background', weights = DataSet_Test2_weight_backgr/test_size )
ax[1,1].hist( predict_ANOMALO2[ y_test2 == 1 ], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{C}^{W}/\Lambda^{2} = 2.0 \times 10^{-6} $', weights = DataSet_Test2_weight_signal/test_size, color = 'green' )
ax[1,1].hist( predict_ANOMALO2[ y_test2 == 2 ], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Modelo Padrão', weights = DataSet_TestSM2_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[1,1].plot( [ best_cut_ANOMALO2,best_cut_ANOMALO2],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut_ANOMALO2 ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut_ANOMALO2 ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[1,1].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut2 = predict_ANOMALO2[ y_test2 == 0 ] > best_cut_ANOMALO2 
n_events_signal_after_cut2 = predict_ANOMALO2[ y_test2 == 1 ] > best_cut_ANOMALO2
n_events_SM_after_cut2 = predict_ANOMALO2[ y_test2 == 2 ] > best_cut_ANOMALO2
n_eventos_Data_after_cut2 = predict_dados[ predict_dados > best_cut_ANOMALO2 ]

print(' --------------- Anomalo 2 --------------- ')
print('Numero de eventos de background depois do corte -->', DataSet_Test2_weight_backgr[ n_events_back_after_cut2 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test2_weight_signal[ n_events_signal_after_cut2 ].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM2_weight_signal[ n_events_SM_after_cut2 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut2 ) )

ax[1,1].set_xlabel( "Probability" )
ax[1,1].set_title( "Mangueira Plot", fontsize = 16  )
ax[1,1].legend( loc = "upper right", fontsize = 15 )
ax[1,1].set_yscale( 'log' )
ax[1,1].text(best_cut_ANOMALO2+0.05 , 2, 'Signal Region \n Blinded Data', color = 'red', fontsize = 16)    
ax[1,1] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[1,1] )

In [ ]:
fig, ax = plt.subplots( 2, 2, figsize=(24,15) )
predict_dados = model_ANOMALO5.predict_proba(X_scaled_dados)[:,1]
bins = 25

hbgt_lgb = ax[0,0].hist( predict_ANOMALO5[ y_test5 == 0 ], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Background', weights = DataSet_Test5_weight_backgr/test_size )
ax[0,0].hist( predict_ANOMALO5[ y_test5 == 1 ], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{0}^{W}/\Lambda^{2}=0.5 \times 10^{-6}$', weights = DataSet_Test5_weight_signal/test_size, color = 'green' )
ax[0,0].hist( predict_ANOMALO5[ y_test5 == 2 ], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Standard Model', weights = DataSet_TestSM5_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[0,0].plot( [ best_cut_ANOMALO5,best_cut_ANOMALO5],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut_ANOMALO5) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut_ANOMALO5 ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[0,0].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut5 = predict_ANOMALO5[ y_test5 == 0 ] > best_cut_ANOMALO5 
n_events_signal_after_cut5 = predict_ANOMALO5[ y_test5 == 1 ] > best_cut_ANOMALO5
n_events_SM_after_cut5 = predict_ANOMALO5[ y_test5 == 2 ] > best_cut_ANOMALO5
n_eventos_Data_after_cut5 = predict_dados[ predict_dados >= best_cut_ANOMALO5 ]

print("----------- Anomalo 5 -----------")
print('Numero de eventos de background depois do corte -->', DataSet_Test5_weight_backgr[ n_events_back_after_cut5 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test5_weight_signal[ n_events_signal_after_cut5 ].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM5_weight_signal[ n_events_SM_after_cut5 ].sum() / test_size )


ax[0,0].text(best_cut_ANOMALO5+0.05 , 6, 'Signal Region \n\n Blinded Data', color = 'red', fontsize = 16)    
ax[0,0].set_ylabel( "Events (Log Scale)" )
ax[0,0].set_title( "Mangueira Plot", fontsize = 16  )
ax[0,0].legend( loc = "upper right", fontsize = 16 )
ax[0,0].set_yscale( 'log' )
ax[0,0] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[0,0] )


predict_dados = model_ANOMALO6.predict_proba(X_scaled_dados)[:,1]
bins = 25

hbgt_lgb = ax[0,1].hist( predict_ANOMALO6[ y_test6 == 0 ], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Data-Driven Background', weights = DataSet_Test6_weight_backgr/test_size )
ax[0,1].hist( predict_ANOMALO6[ y_test6 == 1 ], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{0}^{W}/\Lambda^{2}=1.0 \times 10^{-5}$', weights = DataSet_Test6_weight_signal/test_size, color = 'green' )
ax[0,1].hist( predict_ANOMALO6[ y_test6 == 2 ], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Modelo Padrão', weights = DataSet_TestSM6_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[0,1].plot( [ best_cut_ANOMALO6,best_cut_ANOMALO6],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut_ANOMALO6 ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut_ANOMALO6 ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[0,1].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut6 = predict_ANOMALO6[ y_test6 == 0 ] > best_cut_ANOMALO6 
n_events_signal_after_cut6 = predict_ANOMALO6[ y_test6 == 1 ] > best_cut_ANOMALO6
n_events_SM_after_cut6 = predict_ANOMALO6[ y_test6 == 2 ] > best_cut_ANOMALO6
n_eventos_Data_after_cut6 = predict_dados[ predict_dados >= best_cut_ANOMALO6 ]

print('----------- Anomalo 6 -----------')
print('Numero de eventos de background depois do corte -->', DataSet_Test6_weight_backgr[ n_events_back_after_cut6 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test6_weight_signal[ n_events_signal_after_cut6].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM6_weight_signal[ n_events_SM_after_cut6 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut6 ) )


ax[0,1].text(best_cut_ANOMALO6+0.09 , 0.5, 'Signal Region \n Blinded Data', color = 'red', fontsize = 16)    
ax[0,1].set_title( "Mangueira Plot", fontsize = 16  )
ax[0,1].legend( loc = "upper right", fontsize = 16 )
ax[0,1].set_yscale( 'log' )
ax[0,1] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[0,1] )

predict_dados = model_ANOMALO7.predict_proba(X_scaled_dados)[:,1]
bins = 25

hbgt_lgb = ax[1,0].hist( predict_ANOMALO7[ y_test7 == 0 ], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Data-Driven Background', weights = DataSet_Test7_weight_backgr/test_size )
ax[1,0].hist( predict_ANOMALO7[ y_test7 == 1 ], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 2.0 \times 10^{-6} $', weights = DataSet_Test7_weight_signal/test_size, color = 'green' )
ax[1,0].hist( predict_ANOMALO7[ y_test7 == 2 ], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Modelo Padrão', weights = DataSet_TestSM7_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[1,0].plot( [ best_cut_ANOMALO7,best_cut_ANOMALO7],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut_ANOMALO7 ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut_ANOMALO7 ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[1,0].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut7 = predict_ANOMALO7[ y_test7 == 0 ] >= best_cut_ANOMALO7 
n_events_signal_after_cut7 = predict_ANOMALO7[ y_test7 == 1 ] >= best_cut_ANOMALO7
n_events_SM_after_cut7 = predict_ANOMALO7[ y_test7 == 2 ] >= best_cut_ANOMALO7
n_eventos_Data_after_cut7 = predict_dados[ predict_dados >= best_cut_ANOMALO7 ]

print('----------- Anomalo 7 -----------')
print('Numero de eventos de background depois do corte -->', DataSet_Test7_weight_backgr[ n_events_back_after_cut7 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test7_weight_signal[ n_events_signal_after_cut7].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM7_weight_signal[ n_events_SM_after_cut7 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut7 ) )


ax[1,0].text(best_cut_ANOMALO7+0.05 , 1, 'Signal Region \n Blinded Data', color = 'red', fontsize = 16)    
ax[1,0].set_title( "Mangueira Plot", fontsize = 16  )
ax[1,0].legend( loc = "upper center", fontsize = 14 )
ax[1,0].set_yscale( 'log' )
ax[1,0] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[1,0] )

predict_dados = model_ANOMALO8.predict_proba(X_scaled_dados)[:,1]
bins = 25

hbgt_lgb = ax[1,1].hist( predict_ANOMALO8[ y_test8 == 0 ], bins = np.linspace(0,1,bins), alpha=1, histtype = 'bar', color = 'pink', label = 'Data-Driven Background', weights = DataSet_Test8_weight_backgr/test_size )
ax[1,1].hist( predict_ANOMALO8[ y_test8 == 1 ], bins = np.linspace(0,1,bins), ls='-', linewidth=2.5, histtype = 'step', label = r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 5.0 \times 10^{-6} $', weights = DataSet_Test8_weight_signal/test_size, color = 'green' )
ax[1,1].hist( predict_ANOMALO8[ y_test8 == 2 ], bins = np.linspace(0,1,bins), ls='--', linewidth=2.5, histtype = 'step', label = 'Modelo Padrão', weights = DataSet_TestSM8_weight_signal/test_size, color = 'blue' )

uppery_lgb=np.max(hbgt_lgb[0])
ax[1,1].plot( [ best_cut_ANOMALO8,best_cut_ANOMALO8],[0,uppery_lgb],"-.r", linewidth=2.5,  label='Best cut : {:2.3f}'.format( best_cut_ANOMALO8 ) )
  
counts_1, bin_edges_1 = np.histogram( predict_dados[ predict_dados < best_cut_ANOMALO8 ] , bins = np.linspace(0,1,bins) )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
ax[1,1].errorbar( bin_centres_1, counts_1, yerr=errors_1, xerr=abs(bin_centres_1[1] - bin_centres_1[2])/2, fmt='.', label = 'Data-2016', color = 'black' )

n_events_back_after_cut8 = predict_ANOMALO8[ y_test8 == 0 ] > best_cut_ANOMALO8 
n_events_signal_after_cut8 = predict_ANOMALO8[ y_test8 == 1 ] > best_cut_ANOMALO8
n_events_SM_after_cut8 = predict_ANOMALO8[ y_test8 == 2 ] > best_cut_ANOMALO8
n_eventos_Data_after_cut8 = predict_dados[ predict_dados > best_cut_ANOMALO8 ]

print(' ----------- Anomalo 8 ----------- ')
print('Numero de eventos de background depois do corte -->', DataSet_Test8_weight_backgr[ n_events_back_after_cut8 ].sum() / test_size )
print('Numero de eventos de signal depois do corte -->', DataSet_Test8_weight_signal[ n_events_signal_after_cut8 ].sum() / test_size )
print('Numero de eventos de SM depois do corte -->', DataSet_TestSM8_weight_signal[ n_events_SM_after_cut8 ].sum() / test_size )
print('Numero de eventos de dados depois do corte -->', len( n_eventos_Data_after_cut8 ) )


ax[1,1].set_xlabel( "Probability" )
ax[1,1].set_title( "Mangueira Plot", fontsize = 16  )
ax[1,1].legend( loc = "upper center", fontsize = 16 )
ax[1,1].set_yscale( 'log' )
ax[1,1].text(best_cut_ANOMALO8 + 0.02 , 50, 'Signal Region \n Blinded Data', color = 'red', fontsize = 16, rotation = 90)    
ax[1,1] = hep.cms.label( llabel="Preliminary", rlabel=r"$\mathcal{L} = 9.792\;fb^{-1}$ 13(TeV)", fontsize = 16, ax = ax[1,1] )

In [ ]:
best_cut_ANOMALO1

In [ ]:
y_pred_1 = [np.argmax(line) for line in predict_ANOMALO1 > best_cut_ANOMALO1 ]
y_pred_2 = [np.argmax(line) for line in predict_ANOMALO2 > best_cut_ANOMALO2 ]
y_pred_3 = [np.argmax(line) for line in predict_ANOMALO3 > best_cut_ANOMALO3 ]
y_pred_4 = [np.argmax(line) for line in predict_ANOMALO4 > best_cut_ANOMALO4 ]
y_pred_5 = [np.argmax(line) for line in predict_ANOMALO5 > best_cut_ANOMALO5 ]
y_pred_6 = [np.argmax(line) for line in predict_ANOMALO6 > best_cut_ANOMALO6 ]
y_pred_7 = [np.argmax(line) for line in predict_ANOMALO7 > best_cut_ANOMALO7 ]
y_pred_8 = [np.argmax(line) for line in predict_ANOMALO8 > best_cut_ANOMALO8 ]


In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
import seaborn as sns
def ConfusionMatrix( class_true, class_predict, cmap, title, sample_weight, fmt = 'f' ):
    plt.subplots( figsize=(10,7) )
    conf_mat = multilabel_confusion_matrix( y_true = class_true, y_pred = class_predict, sample_weight = sample_weight)

    sns.heatmap(conf_mat, annot=True, fmt=fmt, cmap = cmap) # fmt = d ou f
    plt.title(title)
    plt.xlabel('Predicted', fontsize = 30 )
    plt.ylabel('Expected' , fontsize = 30 )
    plt.tight_layout()
    plt.show()
    plt.close()

In [ ]:
ConfusionMatrix(y_test8,y_pred_8,'YlGnBu',r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 5.0 \times 10^{-6}$', DataSet_Test8_weight/test_size )

In [ ]:
conf_mat = multilabel_confusion_matrix( y_true = y_test8, y_pred = y_pred_8, sample_weight = DataSet_Test8_weight/test_size )
print(conf_mat)

In [ ]:
y_test8.hist()
plt.yscale('log')

In [ ]:
pd.DataFrame(y_pred_1).hist()
plt.yscale('log')